In [1]:
import csv 
from collections import defaultdict 

with open("/home/estengel/annotator_uncertainty/hit3.0/results/cleaned_data/csv/test_set.csv", "r") as f:
    reader = csv.DictReader(f)
    data = list(reader)
    print(f"test: {len(data)}") 
with open("/home/estengel/annotator_uncertainty/hit3.0/results/cleaned_data/csv/dev_set.csv", "r") as f:
    reader = csv.DictReader(f)
    dev_data = list(reader)
    print(f"dev: {len(dev_data)}")
    data += dev_data



test: 211
dev: 30


## Data stats
- how many examples
- how many answers 

In [2]:
import json 
import numpy as np
n_answers = []
n_examples = 0
n_questions = 0

# how persistent is the repeat problem 
problem = []
for row in data:
    answer_groups = json.loads(row['Answer.answer_groups']) 
    groups_for_row = 0
    answer_group_id_sets = []
    # if row['Input.question_id'] in ["250194002", "223458004", '139390001', '170107002', "176087007", '25515000', '459848003'] :
        # continue
    for group in answer_groups:
        group_ids = [x['id'] + "_" + x['content'] for x in group]
        group_ids = set(group_ids)
        answer_group_id_sets.append(group_ids)
        groups_for_row += len(group_ids)
        n_answers.append(len(group_ids))

    for i, id_set1 in enumerate(answer_group_id_sets):
        for j, id_set2 in enumerate(answer_group_id_sets):
            if i != j and len(id_set1 & id_set2) > 0:
                problem.append(row['Input.question_id']) 

    if groups_for_row > 10:
        print(row['Input.question_id'])
        print(groups_for_row)
        print(row['Answer.answer_groups'])
        for group in answer_groups:
            group_ids = [x['id'] + "_"+ x['content'] for x in group]
            group_ids = set(group_ids)
            print(group_ids)
        break 
    n_questions += len(answer_groups)

    n_examples += 1

problem = list(set(problem)) 
print(f"{len(problem)} repeat examples {problem}")
total_n_answers = sum(n_answers)
min_n_answers = min(n_answers)
max_n_answers = max(n_answers)
mean_n_answers = np.mean(n_answers)

print(f"{total_n_answers} answers in {n_examples} examples, {n_questions} questions")
print(f"min: {min_n_answers}, max: {max_n_answers}, mean: {mean_n_answers}")

0 repeat examples []
1820 answers in 241 examples, 629 questions
min: 1, max: 9, mean: 2.8934817170111287


## How many in pilot were skipped vs non-skipped? 



In [ ]:
skipped = ['445745002', '447840008', '289583000', '265967000', '45836002', '466845001', '47729001', '107886009', '113962006', '12145002', '149739002', '16364001', '172475009', '174667001', '481820007', '217591002', '219173000', '227713000', '230655007', '254210022', '177837002', '261271005', '265051013', '270184001', '279073000', '279241000', '300571000', '303892002']
unskipped = ['456109001', '460694000', '139390001', '165829001', '170107002', '176087007', '321960001', '385276000', '385276000', '25515000', '318850001', '459848003', '297043005', '405041002']

with open("/home/estengel/annotator_uncertainty/hit3.0/csvs/for_mturk_pilot.csv") as f1:
    pilot_data = list(csv.DictReader(f1)) 

qid_list = [str(x) for x in json.loads(pilot_data[0]['questionIDList'])]

skipped_count, unskipped_count = 0, 0
for qid in qid_list:
    if qid in skipped:
        skipped_count += 1
    elif qid in unskipped:
        unskipped_count += 1
    else:
        print(qid)
        raise Exception

print(f"{skipped_count} kskipped, {unskipped_count} unskipped")


## How many skipped in full annotation 

In [ ]:
from pathlib import Path

path = Path("/home/estengel/annotator_uncertainty/hit3.0/results/mturk/")
all_csvs = path.glob("full_hit_round_*/*.csv")


n_skipped = 0
n_annotated = 0
for pp in all_csvs:
    data = list(csv.DictReader(open(pp)))
    for row in data:
        is_skipped = json.loads(row['Answer.is_skip'])
        if is_skipped:
            n_skipped += 1
        else:
            n_annotated += 1
print(f"{n_skipped} skipped, {n_annotated} annotated")
